# Análisis de campañas A/B

El archivo "b08_practica" contiene las siguientes variables de un experimento en el cual se compararon dos campañas ('control' y 'test') para ventas en línea:  
- Campaña: Nombre de la campaña.
- Fecha: Día del registro.
- Gasto: Cantidad gastada en la campaña en dólares.
- Impresiones: Número de impresiones del anuncio a través de la campaña
- Alcance: Número de impresiones únicas recibidas por el anuncio
- Búsquedas: Número de usuarios que realizaron búsquedas en el sitio web.
- Vistas: Número de usuarios que vieron contenido y productos en el sitio web.
- NumCarritos: Número de usuarios que agregaron productos al 'carrito'.
- Compras: Número de compras

Compara las diferentes métricas y aconseja cuál campaña de publicidad es mejor

## Análisis de medias por columna por grupo
Se realiza el análisis de los dos grupos en cada columna. Para cada columna se realizan:
- Tabla pivote con el valor promedio por grupo
- Prueba de normalidad Shapiro-Wilk para cada grupo
- Prueba de homogeneidad de varianza Levene
- Prueba t de diferencia de medias
- Prueba Mann-Whitney de diferencia de medias

In [3]:
# Importación de funciones para las pruebas por realizar
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import levene
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from statsmodels.stats.proportion import proportions_ztest

In [4]:
# Lectura de datos
df = pd.read_excel('data/b08_practica.xlsx')

In [5]:
# Resumen técnico del set de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Campaña      59 non-null     object        
 1   Fecha        59 non-null     datetime64[ns]
 2   Gasto        59 non-null     int64         
 3   Impresiones  59 non-null     int64         
 4   Alcance      59 non-null     int64         
 5   Clicks       59 non-null     int64         
 6   Busquedas    59 non-null     int64         
 7   Vistas       59 non-null     int64         
 8   NumCarritos  59 non-null     int64         
 9   Compras      59 non-null     int64         
dtypes: datetime64[ns](1), int64(8), object(1)
memory usage: 4.7+ KB


In [228]:
# Primeras columnas para familiarización con el contenido de las columnas
df.head()

,Campaña,Fecha,Gasto,Impresiones,Alcance,Clicks,Busquedas,Vistas,NumCarritos,Compras
0,Control,2019-08-01,2280,82702,56930,7016,2290,2159,1819,618
1,Control,2019-08-02,1757,121040,102513,8110,2033,1841,1219,511
2,Control,2019-08-03,2343,131711,110862,6508,1737,1549,1134,372
3,Control,2019-08-04,1940,72878,61235,3065,1042,982,1183,340
4,Control,2019-08-06,3083,109076,87998,4028,1709,1249,784,764


### Gasto

- Hipótesis de normalidad (No rechazada)
- Igualdad de varianzas (No rechazada)
- Prueba de medias (Rechazada: Diferencia significativa entre las medias)

Se detecta un gasto promedio mayor en la campaña de prueba

In [6]:
# Tabla pivote con las medias por grupo
df.groupby('Campaña').agg({'Gasto': 'mean'})

,Gasto
Campaña,
Control,2304.068966
Test,2563.066667


In [230]:
# Prueba de normalidad de los datos de Gasto en campaña: Control
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Control', 'Gasto'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9620704650878906
Valor p: 0.3692897856235504


In [231]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [232]:
# Prueba de normalidad de los datos de Gasto en campaña: Test
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Test', 'Gasto'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9440772533416748
Valor p: 0.11714861541986465


In [233]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [234]:
# Prueba de homogeneidad de varianza de Gasto entre Campañas
stat, p_value = levene(df.loc[df['Campaña'] == 'Control', 'Gasto'],
                       df.loc[df['Campaña'] == 'Test', 'Gasto'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.016724040958384652
Valor p: 0.8975585514162038


In [235]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.


In [236]:
# Prueba t de Gasto entre Campañas
t_statistic, p_value = ttest_ind(df.loc[df['Campaña'] == 'Control', 'Gasto'], 
                                 df.loc[df['Campaña'] == 'Test', 'Gasto'])

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

Estadístico t: -2.793240375594836
Valor p: 0.007093062303374369


In [237]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.


In [238]:
# Realizar la prueba de Mann-Whitney U
statistic, p_value = mannwhitneyu(df.loc[df['Campaña'] == 'Control', 'Gasto'],
                                  df.loc[df['Campaña'] == 'Test', 'Gasto'])

# Imprimir los resultados
print("Estadístico de prueba U:", statistic)
print("Valor p:", p_value)

Estadístico de prueba U: 257.0
Valor p: 0.00711852437564885


In [239]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.


### Impresiones

- Hipótesis de normalidad (No rechazada)
- Igualdad de varianzas (Rechazada: Diferencia significativa entre las varianzas)
- Prueba de medias (Rechazada: Diferencia significativa entre las medias)

Se detecta un número de impresiones promedio mayor en la campaña de control

In [240]:
df.groupby('Campaña').agg({'Impresiones': 'mean'})

,Impresiones
Campaña,
Control,109559.758621
Test,74584.800000


In [241]:
# Prueba de normalidad de los datos de Gasto en campaña: Control
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Control', 'Impresiones'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9425901174545288
Valor p: 0.11722593009471893


In [242]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [243]:
# Prueba de normalidad de los datos de Gasto en campaña: Test
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Test', 'Impresiones'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.948454737663269
Valor p: 0.15366937220096588


In [244]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [245]:
# Prueba de homogeneidad de varianza de Gasto entre Campañas
stat, p_value = levene(df.loc[df['Campaña'] == 'Control', 'Impresiones'],
                       df.loc[df['Campaña'] == 'Test', 'Impresiones'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 6.8867144172541295
Valor p: 0.011125294220972791


In [246]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.


In [247]:
# Prueba t de Gasto entre Campañas
t_statistic, p_value = ttest_ind(df.loc[df['Campaña'] == 'Control', 'Impresiones'], 
                                 df.loc[df['Campaña'] == 'Test', 'Impresiones'])

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

Estadístico t: 4.884544325740239
Valor p: 8.774394114329264e-06


In [248]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.


In [249]:
# Realizar la prueba de Mann-Whitney U
statistic, p_value = mannwhitneyu(df.loc[df['Campaña'] == 'Control', 'Impresiones'],
                                  df.loc[df['Campaña'] == 'Test', 'Impresiones'])

# Imprimir los resultados
print("Estadístico de prueba U:", statistic)
print("Valor p:", p_value)

Estadístico de prueba U: 697.0
Valor p: 7.344126278759323e-05


In [250]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.


### Alcance

- Hipótesis de normalidad (No rechazada)
- Igualdad de varianzas (No rechazada)
- Prueba de medias (Rechazada: Diferencia significativa entre las medias)

Se detecta un alcance promedio mayor en la campaña de control

In [251]:
df.groupby('Campaña').agg({'Alcance': 'mean'})

,Alcance
Campaña,
Control,88844.931034
Test,53491.566667


In [252]:
# Prueba de normalidad de los datos de Gasto en campaña: Control
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Control', 'Alcance'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.973163366317749
Valor p: 0.6480677723884583


In [253]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [254]:
# Prueba de normalidad de los datos de Gasto en campaña: Test
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Test', 'Alcance'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9324314594268799
Valor p: 0.05699457973241806


In [255]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [256]:
# Prueba de homogeneidad de varianza de Gasto entre Campañas
stat, p_value = levene(df.loc[df['Campaña'] == 'Control', 'Alcance'],
                       df.loc[df['Campaña'] == 'Test', 'Alcance'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 2.1186730933639852
Valor p: 0.15099841828036473


In [257]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.


In [258]:
# Prueba t de Gasto entre Campañas
t_statistic, p_value = ttest_ind(df.loc[df['Campaña'] == 'Control', 'Alcance'], 
                                 df.loc[df['Campaña'] == 'Test', 'Alcance'])

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

Estadístico t: 5.300379386681559
Valor p: 1.9462081815900436e-06


In [259]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.


In [260]:
# Realizar la prueba de Mann-Whitney U
statistic, p_value = mannwhitneyu(df.loc[df['Campaña'] == 'Control', 'Alcance'],
                                  df.loc[df['Campaña'] == 'Test', 'Alcance'])

# Imprimir los resultados
print("Estadístico de prueba U:", statistic)
print("Valor p:", p_value)

Estadístico de prueba U: 719.0
Valor p: 1.7202314302800555e-05


In [261]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.


### Búsquedas

- Hipótesis de normalidad (Rechazada: Se dará prioridad a prueba Mann-Whitney)
- Igualdad de varianzas (Rechazada: Diferencia significativa entre las varianzas)
- Prueba de medias (No rechazada)

No hay diferencia significativa en las búsquedas promedio entre campañas.

In [262]:
df.groupby('Campaña').agg({'Busquedas': 'mean'})

,Busquedas
Campaña,
Control,2221.310345
Test,2418.966667


In [263]:
# Prueba de normalidad de los datos de Gasto en campaña: Control
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Control', 'Busquedas'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9283456802368164
Valor p: 0.04985233396291733


In [264]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

Se rechaza la hipótesis nula de normalidad en los datos


In [265]:
# Prueba de normalidad de los datos de Gasto en campaña: Test
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Test', 'Busquedas'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.8963552117347717
Valor p: 0.006848006974905729


In [266]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

Se rechaza la hipótesis nula de normalidad en los datos


In [267]:
# Prueba de homogeneidad de varianza de Gasto entre Campañas
stat, p_value = levene(df.loc[df['Campaña'] == 'Control', 'Busquedas'],
                       df.loc[df['Campaña'] == 'Test', 'Busquedas'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 9.062836553586905
Valor p: 0.0038822267759805146


In [268]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.


In [269]:
# Prueba t de Gasto entre Campañas
t_statistic, p_value = ttest_ind(df.loc[df['Campaña'] == 'Control', 'Busquedas'], 
                                 df.loc[df['Campaña'] == 'Test', 'Busquedas'])

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

Estadístico t: -1.1373340684043094
Valor p: 0.26015715752487034


In [270]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.")

No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.


In [271]:
# Realizar la prueba de Mann-Whitney U
statistic, p_value = mannwhitneyu(df.loc[df['Campaña'] == 'Control', 'Busquedas'],
                                  df.loc[df['Campaña'] == 'Test', 'Busquedas'])

# Imprimir los resultados
print("Estadístico de prueba U:", statistic)
print("Valor p:", p_value)

Estadístico de prueba U: 346.0
Valor p: 0.17964361698819864


In [272]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución")

No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución


### Vistas

- Hipótesis de normalidad (No rechazada)
- Igualdad de varianzas (No rechazada)
- Prueba de medias (No rechazada)

No hay diferencia significativa en las vistas al sitio promedio entre campañas.

In [273]:
df.groupby('Campaña').agg({'Vistas': 'mean'})

,Vistas
Campaña,
Control,1943.793103
Test,1858.000000


In [274]:
# Prueba de normalidad de los datos de Gasto en campaña: Control
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Control', 'Vistas'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9337296485900879
Valor p: 0.06875661015510559


In [275]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [276]:
# Prueba de normalidad de los datos de Gasto en campaña: Test
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Test', 'Vistas'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9403281211853027
Valor p: 0.09282878786325455


In [277]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [278]:
# Prueba de homogeneidad de varianza de Gasto entre Campañas
stat, p_value = levene(df.loc[df['Campaña'] == 'Control', 'Vistas'],
                       df.loc[df['Campaña'] == 'Test', 'Vistas'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.7690131906486748
Valor p: 0.3842035854822884


In [279]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.


In [280]:
# Prueba t de Gasto entre Campañas
t_statistic, p_value = ttest_ind(df.loc[df['Campaña'] == 'Control', 'Vistas'], 
                                 df.loc[df['Campaña'] == 'Test', 'Vistas'])

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

Estadístico t: 0.47615455602474466
Valor p: 0.6357843704297139


In [281]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.")

No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.


In [282]:
# Realizar la prueba de Mann-Whitney U
statistic, p_value = mannwhitneyu(df.loc[df['Campaña'] == 'Control', 'Vistas'],
                                  df.loc[df['Campaña'] == 'Test', 'Vistas'])

# Imprimir los resultados
print("Estadístico de prueba U:", statistic)
print("Valor p:", p_value)

Estadístico de prueba U: 442.0
Valor p: 0.9214899604244585


In [283]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución")

No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución


### NumCarritos

- Hipótesis de normalidad (Rechazada para la campaña de prueba)
- Igualdad de varianzas (No rechazada)
- Prueba de medias (Rechazada: Diferencia significativa entre las medias)

Se detecta un número de carritos promedio mayor en la campaña de control.

In [284]:
df.groupby('Campaña').agg({'NumCarritos': 'mean'})

,NumCarritos
Campaña,
Control,1300.000000
Test,881.533333


In [285]:
# Prueba de normalidad de los datos de Gasto en campaña: Control
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Control', 'NumCarritos'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9554286599159241
Valor p: 0.25248831510543823


In [286]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [287]:
# Prueba de normalidad de los datos de Gasto en campaña: Test
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Test', 'NumCarritos'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9235519766807556
Valor p: 0.03319269046187401


In [288]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

Se rechaza la hipótesis nula de normalidad en los datos


In [289]:
# Prueba de homogeneidad de varianza de Gasto entre Campañas
stat, p_value = levene(df.loc[df['Campaña'] == 'Control', 'NumCarritos'],
                       df.loc[df['Campaña'] == 'Test', 'NumCarritos'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 1.4290971599392581
Valor p: 0.2368617582504474


In [290]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.


In [291]:
# Prueba t de Gasto entre Campañas
t_statistic, p_value = ttest_ind(df.loc[df['Campaña'] == 'Control', 'NumCarritos'], 
                                 df.loc[df['Campaña'] == 'Test', 'NumCarritos'])

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

Estadístico t: 4.24906420944249
Valor p: 8.032960071149043e-05


In [292]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.


In [293]:
# Realizar la prueba de Mann-Whitney U
statistic, p_value = mannwhitneyu(df.loc[df['Campaña'] == 'Control', 'NumCarritos'],
                                  df.loc[df['Campaña'] == 'Test', 'NumCarritos'])

# Imprimir los resultados
print("Estadístico de prueba U:", statistic)
print("Valor p:", p_value)

Estadístico de prueba U: 664.0
Valor p: 0.0005312022228449564


In [294]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución")

Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.


### Compras

- Hipótesis de normalidad (Rechazada para la campaña de prueba)
- Igualdad de varianzas (No rechazada)
- Prueba de medias (No rechazada)

No hay diferencia significativa en las compras promedio entre campañas.

In [295]:
df.groupby('Campaña').agg({'Compras': 'mean'})

,Compras
Campaña,
Control,522.793103
Test,521.233333


In [296]:
# Prueba de normalidad de los datos de Gasto en campaña: Control
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Control', 'Compras'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9381415843963623
Valor p: 0.08964213728904724


In [297]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

No se puede rechazar la hipótesis nula de normalidad en los datos


In [298]:
# Prueba de normalidad de los datos de Gasto en campaña: Test
stat, p_value = shapiro(df.loc[df['Campaña'] == 'Test', 'Compras'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.9181894659996033
Valor p: 0.024077769368886948


In [299]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

Se rechaza la hipótesis nula de normalidad en los datos


In [300]:
# Prueba de homogeneidad de varianza de Gasto entre Campañas
stat, p_value = levene(df.loc[df['Campaña'] == 'Control', 'Compras'],
                       df.loc[df['Campaña'] == 'Test', 'Compras'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 1.230260648051359
Valor p: 0.2720142278811527


In [301]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.


In [302]:
# Prueba t de Gasto entre Campañas
t_statistic, p_value = ttest_ind(df.loc[df['Campaña'] == 'Control', 'Compras'], 
                                 df.loc[df['Campaña'] == 'Test', 'Compras'])

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

Estadístico t: 0.03014479856562245
Valor p: 0.9760568756579724


In [303]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.")

No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.


In [304]:
# Realizar la prueba de Mann-Whitney U
statistic, p_value = mannwhitneyu(df.loc[df['Campaña'] == 'Control', 'Compras'],
                                  df.loc[df['Campaña'] == 'Test', 'Compras'])

# Imprimir los resultados
print("Estadístico de prueba U:", statistic)
print("Valor p:", p_value)

Estadístico de prueba U: 439.0
Valor p: 0.957674926453464


In [305]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución")

No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución


## Análisis de proporciones de conversión

Se realizó también un análisis de proporciones para determinar:
- La efectividad de las campañas para convertir usuarios alcanzados en visitas al sitio
- La efectividad del sitio para convertir visitas en compras

De esta forma, podemos identificar recomendaciones relacionadas a la campaña y al sitio web para aumentar la efectividad de ambos esfuerzos.

In [306]:
# Tabla con proporciones de Visitas, NumCarritos y Compras por grupo
conversion= df.groupby('Campaña').agg({'Gasto': 'sum', 'Alcance': 'sum', 'Vistas': 'sum', 'NumCarritos': 'sum', 'Compras': 'sum'})
conversion

,Gasto,Alcance,Vistas,NumCarritos,Compras
Campaña,,,,,
Control,66818,2576503,56370,37700,15161
Test,76892,1604747,55740,26446,15637


### Conversión de vistas

Se rechaza la hipótesis nula porque hay diferencia significativa entre las proporciones entre campañas. Se detecta un mayor porcentaje de conversión de vistas al sitio en la campaña de prueba a pesar de mostrar un menor alcance.

In [308]:
# Prueba z de proporciones de conversión de vistas
valor_z, p_value  = proportions_ztest(conversion.Vistas, 
                                      conversion.Alcance)

# Imprimir el resultado
print("Valor z:", valor_z)
print("Valor p:", p_value)

Valor z: -79.141183919127
Valor p: 0.0


In [309]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de que hay diferencias significativas en las proporciones.")
else:
    print("No se rechaza la hipótesis nula: no hay suficiente evidencia para afirmar que hay diferencias significativas en las proporciones.")

Se rechaza la hipótesis nula: hay evidencia de que hay diferencias significativas en las proporciones.


In [311]:
# Porcentaje de conversión de vistas al sitio
conversion.Vistas/conversion.Alcance * 100

Campaña
Control    2.187849
Test       3.473445
dtype: float64

### Conversión terminal del sitio

Se rechaza la hipótesis nula porque hay diferencia significativa entre las proporciones entre campañas. Se detecta un mayor porcentaje de conversión de compra de las vistas al sitio de la campaña de prueba a pesar de mostrar un menor alcance inicial.

In [314]:
# Prueba z de proporciones de conversión terminal
valor_z, p_value  = proportions_ztest(conversion.Compras, 
                                      conversion.Vistas)

# Imprimir el resultado
print("Valor z:", valor_z)
print("Valor p:", p_value)

Valor z: -4.3429133988084
Valor p: 1.406055699699901e-05


In [315]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de que hay diferencias significativas en las proporciones.")
else:
    print("No se rechaza la hipótesis nula: no hay suficiente evidencia para afirmar que hay diferencias significativas en las proporciones.")

Se rechaza la hipótesis nula: hay evidencia de que hay diferencias significativas en las proporciones.


In [316]:
# Porcentaje de conversión terminal del sitio
conversion.Compras/conversion.Vistas * 100

Campaña
Control    26.895512
Test       28.053463
dtype: float64